# Usage suggestions for this python package

First install the package:

Activate the python environment you want to use. Go into the folder of this notebook, where setup.py located is, and run 

`pip install -e .`

In [7]:
import relational_image_generation_evaluation as rige

In [8]:
evaluator = rige.Evaluator('ViT-L/14')
# testonly just makes loading faster, to actually test the model you need to set testonly=False
# this is not the test set or so. It's just a flag to make loading faster
dataloader_one = rige.get_one_edge_dataloader(testonly=True)
dataloader_two = rige.get_two_edge_dataloader(testonly=True)
print("len(dataloader_one):", len(dataloader_one))
print("len(dataloader_two):", len(dataloader_two))

Using device cuda:4 for evaluation.
Using text embeddings as input to the model.
Loading filtered test graphs...
Finished loading filtered test graphs
Generating one edge graphs...


100%|██████████| 100/100 [00:00<00:00, 1218.09it/s]


Finished generating one edge graphs
Loading filtered test graphs...
Finished loading filtered test graphs
Generating two edge graphs...


100%|██████████| 100/100 [00:00<00:00, 915.26it/s]

Finished generating two edge graphs
len(dataloader_one): 837
len(dataloader_two): 1076


**The dataloaders give you networkx graphs as they have convenient functionality.**

**You can access all scene information like this (images have to be loaded using PATH_TO_VG/image_id.jpg):**

In [9]:
some_graph = next(iter(dataloader_two))[0]
print("nodes:", some_graph.nodes)
print("edges:", some_graph.edges)
edge = list(some_graph.edges)[0]
print("some edge:", edge, "edge predicate:", some_graph.edges[edge]['predicate'])
node = list(some_graph.nodes)[0]
print("some node:", node, "node label:", some_graph.nodes[node]['name'])
# other information:
print(some_graph.nodes[node].keys())
print(some_graph.edges[edge].keys())
print("image_id:", some_graph.image_id)
print("image dimensions:", some_graph.image_w, some_graph.image_h)
# use this copy function to make a copy of the graph, because much faster than deepcopy and normal copy doesn't copy everything
some_graph_copy = rige.copy_graph(some_graph)

nodes: [1534698, 1534693, 1534701]
edges: [(1534693, 1534698), (1534701, 1534698)]
some edge: (1534693, 1534698) edge predicate: on
some node: 1534698 node label: building
dict_keys(['w', 'h', 'x', 'y', 'attributes', 'name'])
dict_keys(['synsets', 'relationship_id', 'predicate'])
image_id: 102
image dimensions: 800 600


In [10]:
from PIL import Image
images = []
graphs = []
for i in range(10):
    graph = next(iter(dataloader_one))[0]
    image_id = graph.image_id
    # adapt to your local directory
    IMAGE_DIR = '/local/home/jthomm/GraphCLIP/datasets/visual_genome/raw/VG/'
    image = Image.open(IMAGE_DIR + str(image_id) + '.jpg')
    images.append(image)
    graphs.append(graph)

The ViT finetuned models: For each graph you evaluate they give you a relationship probability confidence and an attribute probability confidence. Both are the mean confidences for the relationships and attributes of the graph you give in.

In many graphs, there are no attributes, in this case, 'noattributes' os given back instead of a numerical score. Same behaviour for edges.

In [11]:
scores = evaluator(images,graphs)
print(scores)
print(scores.keys())

{'rel_scores': [0.45698222517967224, 0.9993605017662048, 0.025647636502981186, 0.9703323841094971, 0.9858688116073608, 0.9772343635559082, 0.9066334366798401, 0.9999853372573853, 0.5529768466949463, 0.5258352160453796], 'attr_scores': [0.36680275201797485, 'noattributes', 'noattributes', 0.4471799433231354, 0.08067391067743301, 'noattributes', 'noattributes', 0.5026043057441711, 0.3742857873439789, 0.9566400051116943]}
dict_keys(['rel_scores', 'attr_scores'])
